# Pytorch Practice

In [1]:
import torch
import numpy as np
import matplotlib as plt

# For timer
import time

In [18]:
x = torch.tensor( [5.3, 3])
print( x )

x = x.new_ones(5, 3, dtype=torch.double)
print( x )

# x = torch.rand_like(x, dtype=torch.float)
# print( x )

print( x.size() )
print( x.shape )
print( x+x)

result = torch.zeros(5,3)
torch.add(x,x, out=result)
print("\nResult= \n" + str(result) )
result.add_( torch.rand_like(x))
print("\nResult (in place addition) = \n" + str(result))

tensor([5.3000, 3.0000])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
torch.Size([5, 3])
torch.Size([5, 3])
tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], dtype=torch.float64)

Result= 
tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])

Result (in place addition) = 
tensor([[2.3823, 2.1374, 2.2881],
        [2.3822, 2.0662, 2.3315],
        [2.7880, 2.7541, 2.7067],
        [2.0142, 2.5826, 2.6406],
        [2.2043, 2.9190, 2.9361]])


In [19]:
x = torch.randn(4,4)
print(x)
y = x.view(8,2) # kapalit ng x.reshape sa np/tf
print(y)

tensor([[-1.3491,  1.0726,  0.5579,  0.2897],
        [-0.7006, -0.0612, -1.8959,  0.9481],
        [-1.0205, -0.3404,  1.6546, -0.0442],
        [-1.0541,  2.0118, -0.2435, -0.4460]])
tensor([[-1.3491,  1.0726],
        [ 0.5579,  0.2897],
        [-0.7006, -0.0612],
        [-1.8959,  0.9481],
        [-1.0205, -0.3404],
        [ 1.6546, -0.0442],
        [-1.0541,  2.0118],
        [-0.2435, -0.4460]])


In [20]:
x = torch.rand(2,1)
print(x)
x[1,0].item()

tensor([[0.7989],
        [0.4230]])


0.42301666736602783

In [21]:
# Numpy
a = torch.ones(5)
print( a )

b = a.numpy()
print( b )

a.add_(torch.ones(5) ) # add in place modifies the memory accessed by a and b
print( a )
print( b )

a = torch.add(a, a)
print( a )

c = torch.from_numpy(b)
print( c )

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]
tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]
tensor([4., 4., 4., 4., 4.])
tensor([2., 2., 2., 2., 2.])


In [22]:
# CUDA TEST

x = torch.rand(5,3)
if torch.cuda.is_available():
    device = torch.device("cuda")
    y = torch.ones_like(x, device=device)
    x = x.to(device)
    z = x + y
    print( z ) 
    print(z.to("cpu", torch.double))

tensor([[1.2821, 1.0387, 1.8631],
        [1.1112, 1.0613, 1.2154],
        [1.0653, 1.7064, 1.4318],
        [1.7786, 1.1916, 1.5401],
        [1.3237, 1.2812, 1.5903]], device='cuda:0')
tensor([[1.2821, 1.0387, 1.8631],
        [1.1112, 1.0613, 1.2154],
        [1.0653, 1.7064, 1.4318],
        [1.7786, 1.1916, 1.5401],
        [1.3237, 1.2812, 1.5903]], dtype=torch.float64)


In [23]:
x = torch.ones( 2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [24]:
y = x + 2
print( y )

print( y.grad_fn )

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [25]:
z = y * y * 3
out = z.mean()

print( z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [26]:
out.backward()
print(x.grad)


tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


# Torch NN


In [27]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [28]:
params = list(net.parameters())
print ( len(params) )
print( params[3].shape )

10
torch.Size([16])


In [29]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1287,  0.0117,  0.0369, -0.0985, -0.0120, -0.0466, -0.0334, -0.1315,
          0.0392, -0.0204]], grad_fn=<AddmmBackward>)


In [30]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    input = torch.randn(1, 1, 32, 32)    
    input = input.to(device)
    net.to(device)
    out = net(input)    
    print(out)

tensor([[ 0.1339,  0.0268,  0.0065, -0.1150, -0.0214, -0.0425, -0.0435, -0.1423,
         -0.0018,  0.0036]], device='cuda:0', grad_fn=<AddmmBackward>)


# Autoencoder test on Pytorch


In [3]:
import os

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [92]:



def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

num_epochs = 20
batch_size = 128
learning_rate = 1e-3


def add_noise(img):
    noise = torch.randn(img.size()) * 0.2
    noisy_img = img + noise
    return noisy_img


def plot_sample_img(img, name):
    img = img.view(1, 28, 28)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)


img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(True),
            nn.Linear(256, 64),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(True),
            nn.Linear(256, 28 * 28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

# # Added elapsed timer
# elapsed_start = time.time()
# for epoch in range(num_epochs):
#     epoch_start = time.time()
#     for data in dataloader:
#         img, _ = data
#         img = img.view(img.size(0), -1)
#         noisy_img = add_noise(img)
#         noisy_img = Variable(noisy_img).cuda()
#         img = Variable(img).cuda()
#         # ===================forward=====================
#         output = model(noisy_img)
#         loss = criterion(output, img)
#         MSE_loss = nn.MSELoss()(output, img)
#         # ===================backward====================
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#     # ===================log========================
#     print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
#           .format(epoch + 1, num_epochs, loss.data.item(), MSE_loss.data.item()
#           )
#           )
#     epoch_end = time.time()
#     print("Epoch run time: %.2f" %(epoch_end-epoch_start))
#     if epoch % 10 == 0:
#         x = to_img(img.cpu().data)
#         x_hat = to_img(output.cpu().data)
#         x_noisy = to_img(noisy_img.cpu().data)
#         weights = to_img(model.encoder[0].weight.cpu().data)
#         save_image(x, './mlp_img/x_{}.png'.format(epoch))
#         save_image(x_hat, './mlp_img/x_hat_{}.png'.format(epoch))
#         save_image(x_noisy, './mlp_img/x_noisy_{}.png'.format(epoch))
#         save_image(weights, './filters/epoch_{}.png'.format(epoch))

# elapsed_end = time.time()
# elapsed_total = elapsed_end-elapsed_start
# print("Total elapsed time: %.2f, (in minutes %3d)" %(elapsed_total, (elapsed_total/60)))

# torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/20], loss:0.1164, MSE_loss:0.0358
Epoch run time: 12.65
epoch [2/20], loss:0.0827, MSE_loss:0.0250
Epoch run time: 12.85
epoch [3/20], loss:0.0822, MSE_loss:0.0249
Epoch run time: 12.75
epoch [4/20], loss:0.0670, MSE_loss:0.0201
Epoch run time: 12.78
epoch [5/20], loss:0.0669, MSE_loss:0.0202
Epoch run time: 12.80
epoch [6/20], loss:0.0601, MSE_loss:0.0179
Epoch run time: 12.79
epoch [7/20], loss:0.0587, MSE_loss:0.0177
Epoch run time: 12.77
epoch [8/20], loss:0.0577, MSE_loss:0.0173
Epoch run time: 12.76
epoch [9/20], loss:0.0524, MSE_loss:0.0157
Epoch run time: 12.79
epoch [10/20], loss:0.0559, MSE_loss:0.0168
Epoch run time: 12.81
epoch [11/20], loss:0.0523, MSE_loss:0.0157
Epoch run time: 12.80
epoch [12/20], loss:0.0543, MSE_loss:0.0161
Epoch run time: 12.86
epoch [13/20], loss:0.0552, MSE_loss:0.0164
Epoch run time: 12.77
epoch [14/20], loss:0.0584, MSE_loss:0.0172
Epoch run time: 12.77
epoch [15/20], loss:0.0505, MSE_loss:0.0149
Epoch run time: 12.93
epoch [16/20], loss

In [108]:
model = autoencoder().cuda()
model.load_state_dict(torch.load( './sim_autoencoder.pth' ))

<All keys matched successfully>

In [6]:
model.eval()

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU(inplace=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=256, out_features=784, bias=True)
    (3): Sigmoid()
  )
)

In [107]:
params = list( model.parameters() )
print( params[0].size() )

torch.Size([256, 784])


autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU(inplace=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=256, out_features=784, bias=True)
    (3): Sigmoid()
  )
)